In [1]:
from ipynb.fs.full.setup import *
from ipynb.fs.full.A_matrix import *
from ipynb.fs.full.boundaryCellDisplacement import *
from ipynb.fs.full.boundaryCellTraction import *
from ipynb.fs.full.boundaryPointTraction import *
from ipynb.fs.full.cell_corner_BCs import *
from ipynb.fs.full.boundary_conditions import *

import math
from matplotlib import pyplot as plt

40
4
40
4
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
## Time loop

! mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))

    # Set number of maximum iterations for convergence

    icorr = 1
    maxcorr = 10000000
    fac = 10 ** math.ceil((math.log((A("x").a_P), 10)))

    # Make directory

    # ! mkdir savedVariables/{time}

    residual_array = np.array([])
    moving_average_array = np.array([])

    

    while True:
        # for iteration in [1]:

        U_previous = U_new  # store latest guess values    # change_u new to U

        A_x = A("x").createMatrix()
        A_x, b_x = boundary_conditions(A_x, b_x, U_previous, U_old, U_old_old, "x")

        saveArray("A_x", A_x)

        ## Solve the x-equation

        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        ## Prepare matrix A and b vector for the y-equation

        A_y = A("y").createMatrix()
        A_y, b_y = boundary_conditions(A_y, b_y, U_previous, U_old, U_old_old, "y")

        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        # ## Update U.New with new x and y displacements

        U_new = np.vstack((Ux, Uy)).T

        ## Calculate the residual of each iteration

        normFactor = np.max(U_new)
        residual = math.sqrt(np.mean((U_new - U_previous) ** 2)) / normFactor
        tolerance = 1e-6

        residual_array = np.append(residual_array, residual)

        ##print out residual of each iteration
        if icorr % 100 == 0:
            moving_average = np.mean(
                residual_array[len(residual_array) - 100 : len(residual_array)]
            )
            moving_average_array = np.append(moving_average_array, moving_average)
            print(
                "icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(
                    icorr, residual, normFactor, moving_average, time
                )
            )
            print(tolerance - residual)

        if icorr % 1000 == 0:

            plt.plot(np.arange(0, len(moving_average_array)), moving_average_array)
            plt.hlines(tolerance, 0, len(moving_average_array), color = "C1")
            plt.yscale("log")
            plt.xlabel("Iterations")
            plt.ylabel("Residual")
            plt.show()

        ## Convergence check

        if residual < tolerance:

            print(
                "icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(
                    icorr, residual, normFactor, moving_average, time
                )
            )

            break

        elif icorr > maxcorr:

            break

        icorr = icorr + 1

    ## Update variables

    U_old_old = U_old
    U_old = U_new

    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = U_new[:, 0].reshape(ny + 2, nx + 2)
    u_reshape_y = U_new[:, 1].reshape(ny + 2, nx + 2)

    # #Boundary conditions for u matrix
    U = np.zeros((ny + 2, nx + 2))
    V = np.zeros((ny + 2, nx + 2))

    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y

    # Modulus of displacement

    disp_mod = np.sqrt((U ** 2) + (V ** 2))
    disp_mod_converted = disp_mod * 1e6  # Convert displacement to micrometers

    ### Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U ** 2) + (V ** 2)) * 1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])

fig.show()

print("U graph")
fig = go.Figure(data=[go.Surface(z=U, x=dispx, y=dispy)])

fig.show()

print("V graph")
fig = go.Figure(data=[go.Surface(z=V, x=dispx, y=dispy)])

fig.show()


mkdir: cannot create directory ‘savedVariables’: File exists

Time = 0.0


/home/scottlevie/miniconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


icorr = 100, Residual = 0.00861212273731549842, normFactor = 0.00003249705114585850, moving average = 141922822479031.40625000000000000000, time = 0.00000
-0.0086111227373155
icorr = 200, Residual = 0.00391950111408639629, normFactor = 0.00006476581518580182, moving average = 0.07861821722816901559, time = 0.00000
-0.003918501114086396
icorr = 300, Residual = 0.00241552941697794872, normFactor = 0.00009594279699069960, moving average = 0.04010187889646137094, time = 0.00000
-0.0024145294169779486


KeyboardInterrupt: 

In [ ]:
A("x").a_S*2/(Sfy)

6153846153846.153